# SIESTA 2019, IJM Changes, Simple Query Examples

In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
%load_ext sql
%config SqlMagic.autocommit=False
%config SqlMagic.autopandas=True

In [33]:
pd.set_option('display.max_colwidth', 100)

In [3]:
%sql postgresql://mpinzger@localhost:5432/retrofit

'Connected: mpinzger@retrofit'

In [4]:
%sql \dt change_schema.*

 * postgresql://mpinzger@localhost:5432/retrofit
4 rows affected.


,Schema,Name,Type,Owner
0,change_schema,changes,table,postgres
1,change_schema,commit,table,postgres
2,change_schema,filerevision,table,postgres
3,change_schema,project,table,postgres


## How many different (Java) files have been changed?

In [22]:
%sql SELECT count(distinct filename) FROM change_schema.filerevision;

 * postgresql://mpinzger@localhost:5432/retrofit
1 rows affected.


,count
0,421


## How often did each file change?

In [34]:
%sql SELECT filename, COUNT(*) FROM change_schema.filerevision GROUP BY filename ORDER BY count DESC;

 * postgresql://mpinzger@localhost:5432/retrofit
421 rows affected.


,filename,count
0,retrofit/src/main/java/retrofit/RestAdapter.java,59
1,retrofit/src/main/java/retrofit/RequestBuilder.java,56
2,retrofit/src/main/java/retrofit2/Retrofit.java,38
3,retrofit/src/main/java/retrofit/http/RestAdapter.java,34
4,retrofit/src/main/java/retrofit/RestMethodInfo.java,32
...,...,...
416,sync/src/main/java/retrofit/io/Dirs.java,1
417,retrofit-adapters/rxjava/src/main/java/retrofit2/Result.java,1
418,retrofit-converters/simplexml/src/main/java/retrofit/converter/SimpleXmlConverter.java,1
419,retrofit/src/main/java/retrofit/http/Raw.java,1


## How many single changes does each file have (over all commits)?

In [35]:
%%sql SELECT f.filename, COUNT(changes.id)
FROM change_schema.changes JOIN change_schema.filerevision f on changes.filerevision_id = f.id
GROUP BY filename
ORDER BY count DESC;


 * postgresql://mpinzger@localhost:5432/retrofit
421 rows affected.


,filename,count
0,retrofit/src/main/java/retrofit/RequestBuilder.java,6438
1,retrofit/src/main/java/retrofit2/RequestFactory.java,5562
2,retrofit/src/main/java/retrofit/RestAdapter.java,5146
3,retrofit/src/main/java/retrofit/http/RestAdapter.java,4170
4,retrofit/src/main/java/retrofit2/ServiceMethod.java,4082
...,...,...
416,retrofit/src/main/java/retrofit/http/Body.java,2
417,retrofit-converters/simplexml/src/main/java/retrofit2/converter/simplexml/SimpleXmlResponseBodyC...,2
418,retrofit/src/main/java/retrofit2/BaseUrl.java,1
419,retrofit/src/main/java/retrofit/io/TypedOutput.java,1


## How many single changes PER change type does each file have (over all commits)?

In [38]:
%%sql SELECT f.filename, c.change_type, c.action, COUNT(c.id)
FROM change_schema.changes c JOIN change_schema.filerevision f on c.filerevision_id = f.id
GROUP BY f.filename, c.change_type, c.action 
ORDER BY f.filename, c.change_type, c.action;


 * postgresql://mpinzger@localhost:5432/retrofit
11510 rows affected.


,filename,change_type,action,count
0,android/src/main/java/retrofit/android/AndroidMainThread.java,Argument,INS,8
1,android/src/main/java/retrofit/android/AndroidMainThread.java,Class,INS,2
2,android/src/main/java/retrofit/android/AndroidMainThread.java,Condition,INS,3
3,android/src/main/java/retrofit/android/AndroidMainThread.java,Expression,INS,9
4,android/src/main/java/retrofit/android/AndroidMainThread.java,Field Read,INS,4
...,...,...,...,...
11505,sync/src/main/java/retrofit/io/Dirs.java,Return Type,INS,1
11506,sync/src/main/java/retrofit/io/Dirs.java,Statement,INS,3
11507,sync/src/main/java/retrofit/io/Dirs.java,Target,INS,1
11508,sync/src/main/java/retrofit/io/Dirs.java,Throwable,INS,1


## What are the file revisions for a particular commit?

In [48]:
%%sql SELECT co.commit_dst, co.commit_msg, count(distinct f.id) count_files, count(c.id) count_changes
FROM change_schema.changes c JOIN change_schema.filerevision f ON c.filerevision_id = f.id 
JOIN change_schema.commit co ON f.revision_id = co.id 
GROUP BY co.commit_msg, co.commit_src, co.commit_dst 
ORDER BY count_changes DESC ;

 * postgresql://mpinzger@localhost:5432/retrofit
573 rows affected.


,commit_dst,commit_msg,count_files,count_changes
0,d8fab22ab289f24caf73e2af9d486b92afdad490,Mavenize.\n,38,6150
1,a12554ea1a6b72ec98a079ece934b6e7d25b6518,Break request creation parsing out of service method\n\nThe parsing of annotations which define ...,2,5164
2,2ca195069c02c48d7d862913247e08000e282439,"Introduce call, call adapters, and the response type.\n\n* `Call` is the mechanism for request e...",18,5113
3,06d449fa1e6ebbd81c807fb62dfcde76cf58e7ae,Reorganize project for a final push toward 1.0.\n\n* Bump version to 1.0.0-SNAPSHOT.\n* Conslida...,32,4133
4,c533aab3d23a25f5b5d1b87de0fab1680e897948,"Rename RequestFactoryParser to RequestFactory.Builder.\n\nIt's not perfectly a builder, but it f...",11,3594
...,...,...,...,...
568,d402f551ecaef2a59270559321f5a2ae03323ae6,Make Utils constructor private.\n,1,1
569,f520b68bb40e65bff13fca3102cf93421a5cf2dc,Fix a few tiny issues in the retrofit docs.\n,1,1
570,aa2e0a36d1bf926dade3a92cdca38f6d6ee834b3,Unexpected mime type should be a ConversionException.\n,1,1
571,224a81b1fcf8361a96006e1a47c9c07055329418,Deprecate our SimpleXmlConverterFactory\n,1,1


## What are the file revisions for a particular commit?

In [45]:
%%sql select * from 
change_schema.filerevision f 
join change_schema.commit co ON f.revision_id = co.id 
where co.commit_dst = 'a12554ea1a6b72ec98a079ece934b6e7d25b6518';

 * postgresql://mpinzger@localhost:5432/retrofit
2 rows affected.


,id,filename,revision_id,id,project_id,commit_src,commit_dst,commit_msg,timestamp
0,1119,retrofit/src/main/java/retrofit2/RequestFactory.java,37127,37127,35,8b8887c139c4cf13072499bafe8bb94e06903ea2,a12554ea1a6b72ec98a079ece934b6e7d25b6518,Break request creation parsing out of service method\n\nThe parsing of annotations which define ...,2018-06-13 16:27:27
1,1120,retrofit/src/main/java/retrofit2/ServiceMethod.java,37127,37127,35,8b8887c139c4cf13072499bafe8bb94e06903ea2,a12554ea1a6b72ec98a079ece934b6e7d25b6518,Break request creation parsing out of service method\n\nThe parsing of annotations which define ...,2018-06-13 16:27:27


## What are the changes of a particular commit?

In [62]:
%%sql SELECT c.change_type, c.action, count(c.id) count_changes
FROM change_schema.changes c JOIN change_schema.filerevision f ON c.filerevision_id = f.id 
JOIN change_schema.commit co ON f.revision_id = co.id 
WHERE co.commit_dst = 'a12554ea1a6b72ec98a079ece934b6e7d25b6518' 
AND f.filename like '%/ServiceMethod.java' 
GROUP BY c.change_type, c.action 
ORDER BY count_changes DESC;

 * postgresql://mpinzger@localhost:5432/retrofit
52 rows affected.


,change_type,action,count_changes
0,Statement,DEL,668
1,Variable Read,DEL,404
2,Argument,DEL,362
3,Literal,DEL,156
4,Expression,DEL,138
5,Field Read,DEL,133
6,Condition,DEL,113
7,Variable Type,DEL,105
8,Type Argument,DEL,94
9,Target,DEL,71
